In [1]:
import warnings


warnings.filterwarnings("ignore")

import nltk
from nltk import FreqDist
nltk.download('stopwords')

# !pip install pyLDAvis

import pandas as pd
# pd.set_option("display.max_colwidth", 200)
import numpy as np
import json
import re
import gzip
import spacy

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from gensim.models.coherencemodel import CoherenceModel

from nltk.corpus import stopwords

import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

from nltk.stem import WordNetLemmatizer

from tqdm import tqdm

import gensim
from gensim import corpora

import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


C:\Users\tanma\Anaconda3\lib\site-packages\matplotlib\colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [2]:
import json

In [3]:
from gensim.test.utils import datapath

In [5]:
import random

## Load all the required data

In the real world this will be equivalent to querying a table

In [60]:
inter_df = pd.read_csv('./data/interactions.csv')

In [20]:
engaged_df = pd.read_csv("./data/engaged_df.csv")

In [21]:
engaged_df.head()

,book_id,user_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,clean_reviews
0,5805,4cbecbc15af3db041a8e0f594c642bb5,58f2301bd2d4bbfc1b51e4e5fb161cfe,5,"Remember, remember, the fifth of November. Thi...",Wed Jun 13 17:55:53 -0700 2012,Mon Jan 30 05:58:01 -0800 2017,Thu Jun 14 00:00:00 -0700 2012,NaN,7,0,"['remember', 'remember', 'fifth', 'part', 'quo..."
1,5805,49cc59f1c479d698507627b401d47ecf,761a17f52538341a085b629a316204a1,4,Tinha apontado este livro como um dos que tinh...,Sun Jan 05 03:27:10 -0800 2014,Tue Oct 28 15:58:20 -0700 2014,Tue Oct 28 15:58:20 -0700 2014,NaN,1,0,"['tinha', 'visto', 'portman', 'hora', 'livro',..."
2,5805,5f03864c758bfceb6d7d5e93eeb20044,ac54c3ce0c9f660c03881b0668f79c60,5,Review coming soon! www.youtube.com/ReadTomes,Tue Feb 12 11:38:36 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 00:00:00 -0800 2013,1,0,"['review', 'ing', 'soon', 'readtome']"
3,5805,c309dff1695ed8558b29ea8dcd7479b8,0da0bcc469c2acd15350f9a8f0a74e2b,5,"What better way to celebrate Guy Fawkes Day, t...",Tue May 20 09:52:36 -0700 2014,Thu Mar 16 09:49:11 -0700 2017,Wed Nov 05 14:22:10 -0800 2014,Wed Nov 05 00:00:00 -0800 2014,2,0,"['well', 'way', 'celebrate', 'guy', 'day', 're..."
4,5805,0f6b8c04f811e05c8978bd6b66ce7685,7d670b6c8cac0c086e21ae2f1af6eccb,4,To note - I am writing this review a quarter c...,Tue Jul 03 16:43:26 -0700 2012,Fri Feb 27 14:46:00 -0800 2015,Tue Jan 01 00:00:00 -0800 1991,NaN,10,0,"['note', 'write', 'review', 'quarter', 'centur..."


In [41]:
def to_list(x):
    x = x.strip('][').split(', ')
#     print(x)
    x = [i.strip("'") for i in x]
    return x

In [44]:
engaged_df['clean_reviews'] = engaged_df['clean_reviews'].apply(lambda x: to_list(x))

In [10]:
book_df = pd.read_csv('./data/books_with_topics.csv')

In [48]:
book_df['processed_description'] = book_df['processed_description'].apply(lambda x: to_list(x))

In [ ]:
lda_review_model = gensim.models.ldamodel.LdaModel.load(datapath("./data/best_review_model"))

In [48]:
lda_review_model.print_topics()

[(0,
  '0.027*"book" + 0.011*"read" + 0.010*"life" + 0.009*"story" + 0.008*"make" + 0.007*"one" + 0.006*"would" + 0.006*"like" + 0.006*"time" + 0.005*"school"'),
 (1,
  '0.140*"novel" + 0.133*"graphic" + 0.011*"con" + 0.008*"deadpool" + 0.008*"die" + 0.008*"yang" + 0.007*"kai" + 0.006*"para" + 0.006*"che" + 0.006*"manga"'),
 (2,
  '0.012*"get" + 0.009*"like" + 0.007*"volume" + 0.006*"one" + 0.006*"character" + 0.006*"man" + 0.005*"girl" + 0.005*"make" + 0.005*"thing" + 0.005*"see"'),
 (3,
  '0.024*"story" + 0.014*"batman" + 0.009*"one" + 0.008*"character" + 0.007*"good" + 0.007*"book" + 0.006*"make" + 0.006*"dark" + 0.006*"work" + 0.006*"well"'),
 (4,
  '0.030*"read" + 0.022*"like" + 0.022*"story" + 0.021*"really" + 0.020*"love" + 0.017*"book" + 0.016*"character" + 0.015*"volume" + 0.015*"one" + 0.014*"get"')]

## Get a recommendation for a User

In [4]:
# Let us choose a random user

In [ ]:
c = np.random.choice(engaged_df['user_id'].unique())

In [4]:
def generate_recommendations(c, inter_df, engaged_df, book_df, is_debug = True):
    
    #get all the books the user is reviewing
    r_books = engaged_df[engaged_df['user_id'] == c]['book_id'].unique()
    #get all the books the user has interacted with
    r_books = np.concatenate((r_books , inter_df[inter_df['user_id'] == c]['book_id'].unique()))
    
    if is_debug:
        print("These are the books that the user has interacted with:")
        print()
    #Get the distribution of topics
    book_dist = book_df[book_df['book_id'].isin(r_books)]['dom_topic_reviews'].value_counts() 
    book_dist = book_dist/sum(book_dist)
    
    if is_debug:
        
        print(book_dist)
    
    reco = []
    
    count = 0
    
    #Generate a recommnedation using the LDA model
    #If the user has already interacted with the books, just ignore and try to generate a new recommendation
    
    while count< 10:
        idx = random.choices(book_dist.index, weights=book_dist.values)[0]
        id_reco = np.random.choice(book_df_t[book_df_t['dom_topic_reviews'] == idx].index)
        if is_debug:
            print(id_reco)
        if book_df_t.iloc[id_reco]['book_id'] in engaged_df[engaged_df['user_id'] == c]['book_id']:
            pass
        else:
            reco.append((book_df_t.iloc[id_reco]['title'], idx))
            count = count + 1
    

In [ ]:
recos = generate_recommendations()

In [ ]:
print(recos)